In [257]:
import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from IPython.display import clear_output
import math


Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [29]:
dwf_specs

,Mixed_Rate,ResHD_Rate,ResLD_Rate,Commercial_Rate,Industrial_Rate,Institutional_Rate,Mixed_Population,ResHD_Population,ResLD_Population,Commercial_Area,...,Industrial_Area_Upstream,Institutional_Area_Upstream,Mixed_WaterLoad_Upstream,ResHD_WaterLoad_Upstream,ResLD_WaterLoad_Upstream,Commercial_WaterLoad_Upstream,Industrial_WaterLoad_Upstream,Institutional_WaterLoad_Upstream,Baseflow_WaterLoad_Upstream,Total_WaterLoad_Upstream
Zone,,,,,,,,,,,,,,,,,,,,,
FA2,0.185,0.185,0.185,10.0,10.0,10.0,124.697553,1376.568852,19538.388103,4.578282,...,0.000000,14.067279,23.069047,254.665238,3614.601799,45.782818,0.000000,140.672793,2557.439,6636.230695
GE1,0.250,0.250,0.250,85.0,85.0,85.0,0.000000,91.376044,1544.746896,1.905471,...,0.000000,0.000000,0.000000,22.844011,386.186724,161.965000,0.000000,0.000000,25.920,596.915735
GE2,0.225,0.225,0.225,40.0,40.0,40.0,0.000000,0.000000,160.367532,0.000000,...,0.000000,0.140403,0.000000,22.844011,422.269419,161.965000,0.000000,5.616107,51.840,664.534537
GE3,0.170,0.170,0.170,5.0,5.0,5.0,0.000000,0.000000,1308.709158,3.074690,...,0.000000,1.776500,0.000000,22.844011,850.141921,179.058034,0.000000,10.691648,129.599,1192.334614
GE5,0.170,0.170,0.170,5.0,5.0,5.0,0.000000,0.000000,242.205992,0.000000,...,0.000000,1.776500,0.000000,22.844011,891.316940,179.058034,0.000000,10.691648,198.719,1302.629633
GN14,0.170,0.170,0.170,1.0,1.0,1.0,0.000000,0.000000,1208.187915,1.719584,...,0.000000,0.916640,0.000000,22.844011,627.661364,163.684583,0.000000,6.392345,77.760,898.342303
HL1,0.200,0.200,0.200,10.0,10.0,10.0,0.000000,266.919616,8.662844,1.503518,...,0.000000,46.709032,127.864486,3048.952280,6389.825018,567.881809,0.000000,184.673940,5132.156,15451.353534
HL14,0.175,0.175,0.175,1.0,1.0,1.0,94.911795,4081.698785,10051.044861,2.585835,...,0.000000,27.282697,29.363876,1705.562061,4229.747312,307.040574,0.000000,72.511093,2522.876,8867.100916
HL33,0.250,0.250,0.250,44.0,44.0,44.0,0.000000,0.000000,1199.066100,2.130493,...,0.000000,1.776500,0.000000,22.844011,1191.083465,272.799728,0.000000,10.691648,233.280,1730.698851


In [206]:
#THIS IS A PERMANENT CELL

# model_area = "FSA"
# rolling_average = False

# # summation_csv = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\Summation.csv"
# node_csv = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\MH_Zones.csv"
# dwf_csv = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\DWF_Specs.csv"
# parameter_script = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\Read_Parameters.py"
# outfall_csv = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\Outfall_Summary.csv"

# calibration_sheet = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\Calibration_Master.xlsm"
# rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\FSA_Rainfall_Data_PDT.dfs0"
# model = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter\FSA_North_Base_Backup148.mdb"

# map_folder = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\05. CALIB_RESULTS\Result_Database_FINAL\Report_Maps"
# output_folder = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_148_Jupyter"
# result_folder = r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\05. CALIB_RESULTS\Backup148Runs"

# dfs0_folders = []
# dfs0_folders.append(r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_DATA\FSA_North_Calibration_Data\Gauge")
# dfs0_folders.append(r"J:\SEWER_AREA_MODELS\FSA\FSA_NORTH\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_DATA\FSA_North_Calibration_Data\PS")


model_area = "NSSA"
rolling_average = False
generate_confidence_csvs = True
map_point_spacing = 100

summation_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Summation.csv"
node_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\MH_Zones.csv"
dwf_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\DWF_Specs.csv"
parameter_script = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Read_Parameters.py"
outfall_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Outfall_Summary.csv"

calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Calibration_Specifications.xlsx"
rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\SYSTEM_ASSESSMENT\DATA\RAINFALL\NSSA_Rainfall_Data_PDT.dfs0"
model = r'J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\NSSA_Base.mdb'


map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

In [131]:
#THIS IS A PERMANENT CELL
import subprocess

use_accumulation = True #Must be string to

bat_file_path = output_folder + '\\Read_Parameters.bat'
bat_file = open(bat_file_path, "w")
bat_file.write('"' + parameter_script + '" "' + output_folder + '" "' + model + '" ' + str(use_accumulation))
bat_file.close()

subprocess.call([bat_file_path])


0

In [132]:
#THIS IS A PERMANENT CELL
res_types = ['Mixed','ResHD','ResLD']
ici_types = ['Commercial','Industrial','Institutional']
ww_types = res_types + ici_types

all_types = []
for res_type in res_types:
    all_types.append([res_type,'Population','pe','m3/pe/d'])
for ici_type in ici_types:
    all_types.append([ici_type,'Area','ha','m3/ha/d'])
all_types.append(['Baseflow','','',''])
all_types.append(['Total','','',''])
all_types = pd.DataFrame(all_types, columns =['Load_Type','Description','Unit1','Unit2'])
all_types.set_index('Load_Type',inplace=True)

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

dwf_specs = pd.read_csv(dwf_csv)
dwf_specs.set_index('Zone',inplace=True)

In [204]:
#THIS IS A PERMANENT CELL
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Shift Y (m)']
gauges = gauges[gauges.Model.str.contains(model_area)]
gauges.set_index('Gauge',inplace=True)

periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
report_text = report_text[report_text.Model.str.contains(model_area)]
report_text.fillna('This section was left blank.', inplace=True)

wwf_specs = pd.read_csv(dwf_csv)
wwf_specs.set_index('Zone',inplace=True)
wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')


if generate_confidence_csvs:
    
    map_periods = pd.read_excel(calibration_sheet,sheet_name="Periods_Map")
    map_periods = map_periods.loc[:, :'Model']
    map_periods = map_periods[map_periods.Model.str.contains(model_area)]

    thresholds = pd.read_excel(calibration_sheet,sheet_name="Thresholds",skiprows=1)
    thresholds.rename(columns={'Unnamed: 0':'Color'},inplace=True)
    for column in thresholds.columns[1:]:
        thresholds.rename(columns={column:column + ' ' + thresholds.loc[0,column]},inplace=True)
    for column in thresholds.columns[1:]:
        thresholds.rename(columns={column:column.replace('.1','')},inplace=True)
        thresholds.rename(columns={column:column.replace('.2','')},inplace=True)
    thresholds.drop(index=0,inplace=True)
    thresholds.drop(thresholds.index[3:],inplace=True) 
    thresholds.sort_values(by='DWF Flow Status',ascending=False,inplace=True)
    thresholds.reset_index(inplace=True)

    map_periods_check = map_periods.loc[:, :'Meter Status']
    map_periods_check_count = map_periods_check[map_periods_check['Meter Status']=='Primary']\
        [['Zone','Meter Status']].groupby(['Zone']).count()
    map_periods_check_count.rename(columns={'Meter Status':'Primary Count'},inplace=True)
    map_periods_check = pd.merge(map_periods_check,map_periods_check_count,on=['Zone'],how='left')
    multiple_primary = list(map_periods_check_count[map_periods_check_count['Primary Count']>1].index)

    missing_primary = []
    for zone in list(gauges.Location.unique()):
        if not zone in (map_periods_check_count.index):
            missing_primary.append(zone)

    if len(multiple_primary) > 0 or len(missing_primary) > 0 :
        error_message = 'Please correct the following errors in ' + os.path.basename(calibration_sheet) + ', sheet Periods_Map.\n\n'
        error_message += 'The following zones have multiple primary meters: '  + ','.join(multiple_primary) + '.\n'
        error_message += 'The following zones have no primary meters: '  + ','.join(missing_primary) + '.\n'
        raise ValueError(error_message)


yep


In [124]:
#THIS IS A PERMANENT CELL

#Plan
#1: Import manhole flood and concatenate, similar to other elements
#2: Register Zone to the df
#3: Sum the spill ts by zone.
#4: Add a manhole spill plot to the CSO
#5: Add a top 10 MH spill table
#6: Add a map (nice to have)

node_zones = pd.read_csv(node_csv,dtype={'Node': str})
node_zones.set_index(node_zones.Node,inplace=True)
spill_zones = list(node_zones.Zone.unique())

first_round = True
for f in os.listdir(result_folder):
    if '.ADDOUT.res1d' in f:
        print(f)
        res1d = Res1D(result_folder + '\\' + f)
        flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']
        cover_types = ['Normal','Spilling']

        first_round_in_result = True
        res_zones = set()
        for node in res1d.data.Nodes:
            muid = node.Id
            for i, flood_type in enumerate(flood_types):
                ts = res1d.query.GetNodeValues(muid,flood_type)
                col_name = muid + '-' + cover_types[i]
                if ts != None:
                    if max(ts) > 0 or first_round_in_result == True:
                        spill_df = pd.DataFrame(index=res1d.time_index)
                        spill_df['Node'] = muid
                        zone = node_zones.loc[muid,'Zone']
                        spill_df['Zone'] = zone
                        spill_df['Spill'] = ts
                        spill_df['Spill'] = spill_df['Spill'] * 1000
                        
                        res_zones.add(zone)
                        
                        if first_round == True:
                            spill_df_all = spill_df.copy()
                        else:
                            spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                        first_round = False
                        first_round_in_result = False
                        
        #Create empty df for any zone not represented to show 0 on the graph.
        for spill_zone in spill_zones:
            if not spill_zone in res_zones:
                spill_df = pd.DataFrame(index=res1d.time_index)
                spill_df['Node'] = 'X'
                spill_df['Zone'] = spill_zone
                spill_df['Spill'] = 0           
                spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                                                                                      
spill_df_zones = spill_df_all.copy()
spill_df_zones['Date_Time'] = spill_df_zones.index
spill_df_zones.drop(columns=['Node'],inplace=True)
spill_df_zones = spill_df_zones.groupby(['Zone','Date_Time']).sum()
spill_df_zones.reset_index(inplace=True)

                        
#         flood_stats = []

#         for column in df.columns:
#             if column[-7:]=='-Normal':
#                 muid = column[:-7]
#                 cover_type = 'Normal'
#                 volume = round(max(res1d.query.GetNodeValues(muid,'WaterVolumeAboveGround')),1)     
#             else:
#                 muid = column[:-9]
#                 cover_type = 'Spilling'
#                 volume = round(df[column].sum()*5*60/1000,1)
#             max_flood_discharge = round(df[column].max(),1)
#             flood_stats.append([muid,cover_type,volume,max_flood_discharge])

#         flood_stats = pd.DataFrame(flood_stats,columns=['Node','Cover Type','Spill Volume','Spill Peak'])
#         flood_stats.sort_values(by='Spill Volume',ascending=False,inplace=True)
#         flood_stats.reset_index(inplace=True)
#         flood_stats

FSA_North_DWF_2017-07-04_5d_DWF1.ADDOUT.res1d
FSA_North_DWF_2017-08-01_6d_DWF1.ADDOUT.res1d
FSA_North_DWF_2017-09-13_5d_DWF2.ADDOUT.res1d
FSA_North_DWF_2017-11-15_2d_DWF2.ADDOUT.res1d
FSA_North_WWF_2017-01-16_4d_Jan.ADDOUT.res1d
FSA_North_WWF_2017-03-26_5d_Mar.ADDOUT.res1d
FSA_North_WWF_2017-10-16_5d_Oct.ADDOUT.res1d
FSA_North_WWF_2017-11-18_6d_Nov.ADDOUT.res1d
FSA_North_WWF_2017-11-28_5d_Dec.ADDOUT.res1d
FSA_North_WWF_2019-12-29_6d_2020.ADDOUT.res1d
FSA_North_WWF_2020-01-29_7d_2020.ADDOUT.res1d


KeyError: '7984'

In [125]:
#THIS IS A PERMANENT CELL
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Add outfalls
outfalls = pd.read_csv(outfall_csv,dtype={'Weir': str,'Outfall': str})
outfalls['Res_ID'] = ''
for index, row in outfalls.iterrows():
    prefix = ''
    
    
    if row['Layer'] != 'msm_Link':
        prefix = row['Layer'][4:] + ':'
    muid = prefix + row['Weir']
    
    outfalls.iloc[index,3] = row['Layer'][4:]
    outfalls.iloc[index,4] = muid
     
    print(muid)
    flow_pipes.append(muid)
    
#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)


Weir:11E
Weir:13W
Weir:15W
Weir:16W_New
Weir:17W
Weir:1F
Weir:1G
Weir:1W
Weir:20th Street PS Overflow
Weir:2F
Weir:2G
Weir:2S
Weir:3F
Weir:3S
Weir:3W
Weir:4F
Weir:4S
Weir:52121 Port Coquitlam Weir
Weir:5BE
Weir:6E
Weir:8AW
Weir:Baynes Overflow
Orifice:Braid Street Gate
Weir:Cliff Ave N Intercept
Weir:Cliff Ave N Wet Well Overflow
Orifice:KatzieDuckBill
Link_156_FSA_S
Link_177_FSA_S
Weir:marshend
Weir:Maryhill Overflow Weir
Weir:NS4-SSO
Orifice:NW CSO Tank Overflow Gate
Orifice:Orifice_1
Weir:ROYAL AVEPS OUTFALL
Weir:Short St
Weir:Slaughterhouse
Weir:SSO weir
Valve:Valve_2


In [367]:
from mikeio.dfs0 import Dfs0
dfs0_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS"

dfs = Dfs0()
res = mikeio.read(dfs0_folder + '\\' 'NV13_20180628_20190410_5min.dfs0')
df = res.to_dataframe()
df.iloc[:,[0,6]]

res.items

[NV13_Flow_L <Discharge> (liter per sec),
 NV13_Flow_M <Discharge> (meter pow 3 per sec),
 GWI <Discharge> (meter pow 3 per sec),
 Discharge_Minus_GWI <Discharge> (meter pow 3 per sec),
 VW14_Rainfall <Rainfall> (millimeter),
 NV13-Depth <Water Depth> (meter),
 NV13_Level <Water Level> (meter),
 NV13-Velocity <Flow velocity> (meter per sec)]

In [408]:
str(res.items[0])[-27:]




'<Discharge> (liter per sec)'

In [388]:
warnings

[]

In [409]:
#Import rain gauges

rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()
warnings = []

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':

            gauge_id = re.split(r'_|[.]',f)[0]
            res = mikeio.read(dfs0_folder + '\\' + f)
            ts = res.to_dataframe()

            gauge_ids.append(gauge_id)
            ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
            if str(res.items[0])[-33:] == '<Discharge> (meter pow 3 per sec)':
                ts.Flow = ts.Flow * 1000
            elif str(res.items[0])[-27:] != '<Discharge> (liter per sec)':
                warnings.append(f + ' item 1 not a flow type')                                                   
            if len(ts.columns)>6:
                ts.rename(columns={ts.columns[6]:'Level'},inplace=True)
                if str(res.items[6])[-21:] != '<Water Level> (meter)':
                    warnings.append(f + ' item 6 not a level type') 
            else:
                ts['Level'] = np.nan

            if len(ts.columns)>7:
                ts.rename(columns={ts.columns[7]:'Velocity'},inplace=True) 
                if str(res.items[7])[-31:] != '<Flow velocity> (meter per sec)':
                    warnings.append(f + ' item 6 not a velocity type') 
            else:
                ts['Velocity'] = np.nan
            ts['Gauge'] = gauge_id
            ts = ts[['Gauge','Flow','Level','Velocity']]
            ts['Seconds'] = ts.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
            ts['Volume'] = ts.Flow * ts.Seconds / 1000
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
    
for warning in warnings:
    print (warning)
    
measured[measured.Gauge=='HL33']
        

,Gauge,Flow,Level,Velocity,Seconds,Volume
2018-05-24 00:00:00,HL33,42.830002,59.799198,1.595,300.0,12.849001
2018-05-24 00:05:00,HL33,30.655001,59.783901,1.450,300.0,9.196500
2018-05-24 00:10:00,HL33,23.020000,59.774399,1.295,300.0,6.906000
2018-05-24 00:15:00,HL33,17.850000,59.765999,1.190,300.0,5.355000
2018-05-24 00:20:00,HL33,15.345000,59.759201,1.210,300.0,4.603500
...,...,...,...,...,...,...
2019-04-03 23:35:00,HL33,10.434500,59.748501,1.095,300.0,3.130350
2019-04-03 23:40:00,HL33,39.360001,59.786301,1.510,300.0,11.808000
2019-04-03 23:45:00,HL33,60.404999,59.814899,1.830,300.0,18.121500
2019-04-03 23:50:00,HL33,46.360001,59.799500,1.715,300.0,13.908000


In [396]:
measured[measured.Gauge=='NVLS1']

,Gauge,Flow,Level,Velocity,Seconds,Volume
2018-10-19 10:00:00,NVLS1,NaN,NaN,NaN,-43077300.0,NaN
2018-10-19 10:05:00,NVLS1,NaN,NaN,NaN,300.0,NaN
2018-10-19 10:10:00,NVLS1,NaN,NaN,NaN,300.0,NaN
2018-10-19 10:15:00,NVLS1,NaN,NaN,NaN,300.0,NaN
2018-10-19 10:20:00,NVLS1,NaN,NaN,NaN,300.0,NaN
...,...,...,...,...,...,...
2020-02-29 23:35:00,NVLS1,0.084204,36.421101,NaN,300.0,0.025261
2020-02-29 23:40:00,NVLS1,0.098973,36.396301,NaN,300.0,0.029692
2020-02-29 23:45:00,NVLS1,0.081430,36.416199,NaN,300.0,0.024429
2020-02-29 23:50:00,NVLS1,0.100488,36.393101,NaN,300.0,0.030146


In [39]:
# rid = 'Weir:MKO_Overflow'
# if rid in flow_pipes or rid in list(summation_df.MUID):
#     print ("Importing pipe " + rid + " discharge at " + str(dt.datetime.now()))
#     for di in reach.DataItems:
#         if di.Quantity.Id.lower() == 'discharge':
#             pass

#             values = list(di.CreateTimeSeriesData(0))
#             flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
#             flow_df['ResultFile'] = f
#             flow_df['MUID'] = rid
#             flow_df['DateTimeRef'] = flow_df.index
#             flow_df['Discharge'] = values                          
#             flow_df['Discharge'] = flow_df['Discharge'] * 1000
            
# flow_df

In [40]:
results = []
first_level = True
first_flow = True
first_velocity = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f:
        resultdata = Res1D_v5(result_folder + '\\' + f)
        reaches = resultdata.resultData.Reaches
        nodes = resultdata.resultData.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))


        for i, node in enumerate(nodes):
            
            rid = node.Id
            if rid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                for di in node.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        values = list(di.CreateTimeSeriesData(0))
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
        
        
        first_round = True
        for i, reach in enumerate(reaches):
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count
            levelpoints = (gridpoints + 1)/2

            if rid in us_level_pipes or rid in ds_level_pipes:
            
                print ("Importing pipe " + reach.Id + " level at " + str(dt.datetime.now()))
                print (gridpoints)
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        if rid in us_level_pipes: 
                            values = list(di.CreateTimeSeriesData(0))
                        else:
                            values = list(di.CreateTimeSeriesData(levelpoints - 1))
                            
                        
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
#                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
                        
                        
            
        for i, reach in enumerate(reaches):
            
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
#             if 'Weir:' in rid:
#                 rid = rid[5:]
#             if 'Orifice:' in rid:
#                 rid = rid[8:]
            gridpoints = reach.GridPoints.Count
            if rid in flow_pipes or rid in list(summation_df.MUID):
                print ("Importing pipe " + reach.Id + " discharge at " + str(dt.datetime.now()))
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'discharge':

                        values = list(di.CreateTimeSeriesData(0))
                        flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        flow_df['ResultFile'] = f
                        flow_df['MUID'] = rid
                        flow_df['DateTimeRef'] = flow_df.index
                        flow_df['Discharge'] = values                          
                        flow_df['Discharge'] = flow_df['Discharge'] * 1000
                        
                        flow_df['Seconds'] = flow_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                        flow_df['Volume'] = flow_df.Discharge * flow_df.Seconds / 1000
                        flow_df.drop(columns=['Seconds'],inplace=True)

                        if rolling_average == True:
                            flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()

                        
                        if first_flow == True:
                            flow_df_all = flow_df.copy()

                        else:
                            flow_df_all = pd.concat([flow_df_all,flow_df])
                        first_flow = False
                        
        for i, reach in enumerate(reaches):
            
            rid = reach.Id[:(-1 * len(str(i)) - 1)]

            gridpoints = reach.GridPoints.Count
            if rid in flow_pipes:
                print ("Importing pipe " + reach.Id + " velocity at " + str(dt.datetime.now()))
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'flowvelocity':

                        values = list(di.CreateTimeSeriesData(0))
                        velocity_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        velocity_df['ResultFile'] = f
                        velocity_df['MUID'] = rid
                        velocity_df['DateTimeRef'] = velocity_df.index
                        velocity_df['Velocity'] = values                          

                        
                        if first_velocity == True:
                            velocity_df_all = velocity_df.copy()
                        else:
                            velocity_df_all = pd.concat([velocity_df_all,velocity_df])
                        first_velocity = False

        first_round = False
        
#Summation
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum','Volume':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)

flow_df_all = pd.concat([flow_df_all,df_result_sum])
# flow_df_all['Seconds'] = flow_df_all.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
# flow_df_all['Volume'] = flow_df_all.Discharge * flow_df_all.Seconds / 1000


Importing network NSSA_DWF_Cal_2018-07-26_Base.res1d at 2022-11-21 15:37:54.201103
Importing node 10497 at 2022-11-21 15:37:54.201103
Importing node 10498 at 2022-11-21 15:37:54.204132
Importing node 1_119 at 2022-11-21 15:37:54.208135
Importing node 6071 at 2022-11-21 15:37:54.211163
Importing node 6124 at 2022-11-21 15:37:54.215167
Importing node 6819 at 2022-11-21 15:37:54.219171
Importing node 6822 at 2022-11-21 15:37:54.222173
Importing node 7133 at 2022-11-21 15:37:54.226217
Importing node 7169 at 2022-11-21 15:37:54.231220
Importing node 7317 at 2022-11-21 15:37:54.234254
Importing node 9488 at 2022-11-21 15:37:54.237255
Importing node 9525 at 2022-11-21 15:37:54.240258
Importing node 9551 at 2022-11-21 15:37:54.243290
Importing node 9561 at 2022-11-21 15:37:54.247294
Importing node 9600 at 2022-11-21 15:37:54.250323
Importing node 9619 at 2022-11-21 15:37:54.253324
Importing node 9624 at 2022-11-21 15:37:54.256327
Importing node 9629 at 2022-11-21 15:37:54.259330
Importing node

Importing pipe 49582-903 discharge at 2022-11-21 15:37:55.504067
Importing pipe 49592-913 discharge at 2022-11-21 15:37:55.510073
Importing pipe 49598-919 discharge at 2022-11-21 15:37:55.515078
Importing pipe 49640-950 discharge at 2022-11-21 15:37:55.529090
Importing pipe 49652-962 discharge at 2022-11-21 15:37:55.534095
Importing pipe 49709-1018 discharge at 2022-11-21 15:37:55.539099
Importing pipe 49727-1036 discharge at 2022-11-21 15:37:55.545105
Importing pipe 50870-1397 discharge at 2022-11-21 15:37:55.553112
Importing pipe 50924-1435 discharge at 2022-11-21 15:37:55.559146
Importing pipe 50932-1443 discharge at 2022-11-21 15:37:55.564177
Importing pipe 50933-1444 discharge at 2022-11-21 15:37:55.569182
Importing pipe 50940-1451 discharge at 2022-11-21 15:37:55.574213
Importing pipe 50952-1463 discharge at 2022-11-21 15:37:55.579218
Importing pipe 54280-1513 discharge at 2022-11-21 15:37:55.585223
Importing pipe 54287-1519 discharge at 2022-11-21 15:37:55.590266
Importing pipe 

Importing pipe 43945-350 velocity at 2022-11-21 15:37:56.828886
Importing pipe 43946-351 velocity at 2022-11-21 15:37:56.832890
Importing pipe 43953-358 velocity at 2022-11-21 15:37:56.836894
Importing pipe 43987-390 velocity at 2022-11-21 15:37:56.840897
Importing pipe 43989-392 velocity at 2022-11-21 15:37:56.844901
Importing pipe 43999-402 velocity at 2022-11-21 15:37:56.848905
Importing pipe 47391-580 velocity at 2022-11-21 15:37:56.853909
Importing pipe 47438-627 velocity at 2022-11-21 15:37:56.857913
Importing pipe 49559-884 velocity at 2022-11-21 15:37:56.862917
Importing pipe 49580-901 velocity at 2022-11-21 15:37:56.866921
Importing pipe 49582-903 velocity at 2022-11-21 15:37:56.869924
Importing pipe 49592-913 velocity at 2022-11-21 15:37:56.874928
Importing pipe 49598-919 velocity at 2022-11-21 15:37:56.879933
Importing pipe 49640-950 velocity at 2022-11-21 15:37:56.883937
Importing pipe 49652-962 velocity at 2022-11-21 15:37:56.888941
Importing pipe 49709-1018 velocity at 20

Importing network NSSA_WWF_Cal_2018-11-25_Base.res1d at 2022-11-21 15:37:58.716971
Importing node 10497 at 2022-11-21 15:37:58.716971
Importing node 10498 at 2022-11-21 15:37:58.721975
Importing node 1_119 at 2022-11-21 15:37:58.725979
Importing node 6071 at 2022-11-21 15:37:58.731011
Importing node 6124 at 2022-11-21 15:37:58.735013
Importing node 6819 at 2022-11-21 15:37:58.740018
Importing node 6822 at 2022-11-21 15:37:58.746060
Importing node 7133 at 2022-11-21 15:37:58.750078
Importing node 7169 at 2022-11-21 15:37:58.765174
Importing node 7317 at 2022-11-21 15:37:58.769178
Importing node 9488 at 2022-11-21 15:37:58.773222
Importing node 9525 at 2022-11-21 15:37:58.777226
Importing node 9551 at 2022-11-21 15:37:58.783231
Importing node 9561 at 2022-11-21 15:37:58.787275
Importing node 9600 at 2022-11-21 15:37:58.791279
Importing node 9619 at 2022-11-21 15:37:58.795313
Importing node 9624 at 2022-11-21 15:37:58.799316
Importing node 9629 at 2022-11-21 15:37:58.809366
Importing node

Importing pipe 43945-350 discharge at 2022-11-21 15:38:00.076428
Importing pipe 43946-351 discharge at 2022-11-21 15:38:00.083434
Importing pipe 43953-358 discharge at 2022-11-21 15:38:00.090441
Importing pipe 43987-390 discharge at 2022-11-21 15:38:00.097447
Importing pipe 43989-392 discharge at 2022-11-21 15:38:00.104454
Importing pipe 43999-402 discharge at 2022-11-21 15:38:00.110459
Importing pipe 47391-580 discharge at 2022-11-21 15:38:00.118467
Importing pipe 47438-627 discharge at 2022-11-21 15:38:00.125506
Importing pipe 49559-884 discharge at 2022-11-21 15:38:00.134543
Importing pipe 49580-901 discharge at 2022-11-21 15:38:00.141549
Importing pipe 49582-903 discharge at 2022-11-21 15:38:00.147585
Importing pipe 49592-913 discharge at 2022-11-21 15:38:00.154621
Importing pipe 49598-919 discharge at 2022-11-21 15:38:00.160625
Importing pipe 49640-950 discharge at 2022-11-21 15:38:00.167657
Importing pipe 49652-962 discharge at 2022-11-21 15:38:00.174692
Importing pipe 49709-1018

Importing pipe 56296-1593 discharge at 2022-11-21 15:38:01.529355
Importing pipe 56393-1597 discharge at 2022-11-21 15:38:01.536361
Importing pipe 56394_3-1600 discharge at 2022-11-21 15:38:01.544368
Importing pipe 56401-1605 discharge at 2022-11-21 15:38:01.551375
Importing pipe 56405-1606 discharge at 2022-11-21 15:38:01.558381
Importing pipe Link_15-1612 discharge at 2022-11-21 15:38:01.565388
Importing pipe Weir:MKO_Overflow-1635 discharge at 2022-11-21 15:38:01.572394
Importing pipe Orifice:LynnSiphonBlowDownOrifice-1670 discharge at 2022-11-21 15:38:01.579400
Importing pipe 40472-21 velocity at 2022-11-21 15:38:01.586448
Importing pipe 41250-74 velocity at 2022-11-21 15:38:01.591451
Importing pipe 41295-120 velocity at 2022-11-21 15:38:01.596456
Importing pipe 41304-129 velocity at 2022-11-21 15:38:01.600459
Importing pipe 43611-209 velocity at 2022-11-21 15:38:01.605464
Importing pipe 43648-245 velocity at 2022-11-21 15:38:01.610468
Importing pipe 43945-350 velocity at 2022-11-2

Importing pipe 43946-351 velocity at 2022-11-21 15:38:03.026750
Importing pipe 43953-358 velocity at 2022-11-21 15:38:03.032724
Importing pipe 43987-390 velocity at 2022-11-21 15:38:03.039731
Importing pipe 43989-392 velocity at 2022-11-21 15:38:03.044735
Importing pipe 43999-402 velocity at 2022-11-21 15:38:03.050741
Importing pipe 47391-580 velocity at 2022-11-21 15:38:03.057747
Importing pipe 47438-627 velocity at 2022-11-21 15:38:03.062752
Importing pipe 49559-884 velocity at 2022-11-21 15:38:03.069758
Importing pipe 49580-901 velocity at 2022-11-21 15:38:03.075764
Importing pipe 49582-903 velocity at 2022-11-21 15:38:03.080768
Importing pipe 49592-913 velocity at 2022-11-21 15:38:03.086774
Importing pipe 49598-919 velocity at 2022-11-21 15:38:03.092816
Importing pipe 49640-950 velocity at 2022-11-21 15:38:03.097821
Importing pipe 49652-962 velocity at 2022-11-21 15:38:03.103826
Importing pipe 49709-1018 velocity at 2022-11-21 15:38:03.109832
Importing pipe 49727-1036 velocity at 2

Importing pipe Link_15-1612 velocity at 2022-11-21 15:38:04.698101
Importing pipe Weir:MKO_Overflow-1635 velocity at 2022-11-21 15:38:04.705108
Importing pipe Orifice:LynnSiphonBlowDownOrifice-1670 velocity at 2022-11-21 15:38:04.706108


In [41]:
# effluent_types = ['Com','st','in','Henrik','Logan']
# for i, effluent_type in enumerate(effluent_types):
#     if i == 0:
#         effluent_type_string = effluent_type
#     elif i == len(effluent_types)-1:
#         effluent_type_string += ' and ' + effluent_type
#     else:
#         effluent_type_string += ', ' + effluent_type

# effluent_type_string

In [42]:
qs = set()
for i, reach in enumerate(reaches):
    for di in reach.DataItems:
        qs.add(di.Quantity.Id)
qs

{'ControlStrategyId',
 'CrestLevel',
 'Discharge',
 'DischargeInStructure',
 'FlowVelocity',
 'GateLevel',
 'WaterLevel'}

In [43]:
result_outfalls_all_filter

NameError: name 'result_outfalls_all_filter' is not defined

In [ ]:
fig = go.Figure()
for outfall in list(outfalls_filter.Outfall):
    
    result_outfalls_all_filter = result_outfalls_all[result_outfalls_all.Outfall==outfall]
    
    fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                 y = result_outfalls_all_filter.Discharge, 
                                 mode='lines',name=outfall))

fig.update_layout(title = 'CSO/SSO in ' + zone)
fig.show()

In [ ]:
# zone = 'NV1'
# has_outfalls = False
# if zone in list(outfalls.Zone):
#     has_outfalls = True
#     outfalls_filter = outfalls[outfalls.Zone==zone].copy()
#     outfalls_filter.reset_index(inplace=True)
    
#     for index, row in outfalls_filter.iterrows():
#         pipe = row['Res_ID']
#         result_outfalls = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
#         result_outfalls['Outfall'] = row['Outfall']
#         if index == 0:
#             result_outfalls_all = result_outfalls.copy()
#         else:
#             result_outfalls_all = pd.concat([result_outfalls_all,result_outfalls])
    
#         print(index)
# result_outfalls_all

In [86]:
zone = 'HL14'

spill_df_all_zone = spill_df_all.loc[(spill_df_all.Zone==zone) & (spill_df_all.index >= start) & (spill_df_all.index <= end)].copy()
spill_df_all_zone.drop(columns=['Zone'],inplace=True)
spill_top_ten = spill_df_all_zone.groupby(['Node']).max()
spill_top_ten.reset_index(inplace=True)
spill_top_ten.sort_values(by='Spill',ascending=False,inplace=True)
spill_top_ten.reset_index(drop=True,inplace=True)
spill_top_ten = spill_top_ten[spill_top_ten['Spill']>0]
spill_top_ten = spill_top_ten.head(10)
spill_top_ten

,Node,Spill


In [103]:
error_list = []
for gauge in gauge_ids:
    
#      if gauge == 'HL14':
        

        location_type = gauges.loc[gauge,'Location Type']
        zone = str(gauges.loc[gauge,'Location'])
        pipe = str(gauges.loc[gauge,'Pipe'])
        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
        rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
        MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
        NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
        MHName = str(gauges.loc[gauge,'Node1 AssetName']) 
        number_catchments = wwf_specs.loc[zone,'Number sanitary'] \
            + wwf_specs.loc[zone,'Number combined'] \
            + wwf_specs.loc[zone,'Number stormwater']
        
        effluent_types = []
        if wwf_specs.loc[zone,'Number sanitary'] > 0:
            effluent_types.append('Sanitary')
        if wwf_specs.loc[zone,'Number combined'] > 0:
            effluent_types.append('Combined')
        if wwf_specs.loc[zone,'Number stormwater'] > 0:
            effluent_types.append('Storm')
        
        for i, effluent_type in enumerate(effluent_types):
            if i == 0:
                effluent_type_string = effluent_type
            elif i == len(effluent_types)-1:
                effluent_type_string += ' and ' + effluent_type
            else:
                effluent_type_string += ', ' + effluent_type          
            
            

        
        local_population = 0
        for ww_type in ww_types:
            if ww_type in res_types:                
                local_population += dwf_specs.loc[zone,ww_type + '_Population']
                
        
        total_area = round(wwf_specs.loc[zone,'Drainage area (ha)'],1)

        PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
        PipeDimension = str(round(gauges.loc[gauge, 'Pipe Dimension'],2))


        with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
            
            f.write('<style>')
#             f.write('@page {margin-left: 5cm;}')
            f.write('p    {width:1000px;}')
            f.write('p    { font-size: 20px;}')
#---------------------------------------------------------------------------------------------------------------------        
 
    
            f.write('* {\n')
            f.write('  box-sizing: border-box;\n')
            f.write('}\n')

            f.write('.row {\n')
            f.write('  margin-left:0px;\n')
            f.write('  margin-right:-5px;\n')

            f.write('}\n')

            f.write('.column {\n')
            f.write('  float: left;\n')
            f.write('  width: 17%;\n')
            f.write('  padding: 5px;\n')
            f.write('}\n')

            f.write('/* Clearfix (clear floats) */\n')
            f.write('.row::after {\n')
            f.write('  content: "";\n')
            f.write('  clear: both;\n')
            f.write('  display: table;\n')
            f.write('}\n')

            f.write('table.first, table.first th, table.first td {\n')
            f.write('  border: 2px solid;\n')
            f.write('  padding: 4px;\n')

            f.write('  border-collapse: collapse;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 800px;\n')
            f.write('}\n') 

            f.write('table.second {\n')
            f.write('  border: 2px solid;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 310px;\n')
            f.write('  padding: 1px;\n')
            f.write('  margin: 1px;\n')

            f.write('})\n')    
    
#---------------------------------------------------------------------------------------------------------------------        
             
#             f.write('table.grid, th, td {')
#             f.write('  border: 1px solid;')
#             f.write('  padding: 4px;')
#             f.write('  border-collapse: collapse;')
            
            f.write('})')            
            f.write('</style>')
            f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                    + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')

            f.write('<h2>1. Description of the Calibration Area</h2>')
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>Number of Catchments</td>\n')
            f.write('<td>'+ str(number_catchments) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Zone Area (Ha)</td>\n')
            f.write('<td>'+ str(total_area) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Local Residential Population</td>\n')
            f.write('<td>'+ str(int(local_population)) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Effluent Type</td>\n')
            f.write('<td>'+ effluent_type_string +'</td>\n')
            f.write('</tr>\n')            
            for ww_type in ww_types:
                if ww_type not in res_types:
                    f.write('<tr>\n')
                    f.write('<td>' + ww_type + ' Area (Ha)</td>\n')
#                     f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_' + description],1)) +'</td>\n')
                    f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_Area'],1)) +'</td>\n')
                    f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('</table>')
            
            f.write('<h2>2. Description of the Sensor and Network</h2>')            
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Monitoring Type' +'</td>\n')
            f.write('<td>'+ MonitoringType +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Node ID' +'</td>\n')
            f.write('<td>'+ node +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe ID' +'</td>\n')
            f.write('<td>'+ pipe +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Shape' +'</td>\n')
            f.write('<td>'+ PipeShape +'</td>\n')
            f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
            f.write('<td>'+ PipeDimension +'</td>\n')
            f.write('</tr>\n')
            f.write('</table>')
    
            
            f.write('<h2>3. Model Calibration Setup: </h2>')
                    
            f.write('<table class="first">\n')
            f.write('  <tr>\n')
            f.write('    <th>Simulation</th>\n')
            f.write('    <th>Start Date</th>\n')
            f.write('    <th>End Date</th>\n')
            f.write('    <th>Duration</th>\n')
            f.write('  </tr>\n')

            for period_spec in period_specs:
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                duration = str((end - start).days) + ' days'
                f.write('  <tr>\n')
                f.write('    <td>' + description + '</th>\n')
                f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + duration + '</th>\n')
                f.write('  </tr>\n')                   
            f.write('</table>\n')
                                      
            f.write('<h2>4. Zone Map</h2>\n')
            f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">\n')
            f.write('<h2>5. DWF Calibration Summary</h2>\n')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>\n')
            f.write('<h2>6. WWF Calibration and Validation Summary</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>\n')
            f.write('<h2>7. Calibration Issues</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>\n')
            f.write('<h2>8. Recommendations</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>\n')
            f.write('<h2>9. DWF Calibration Plots and Statistics</h2>\n')

            wwf_plot_header_added = False
            for period_spec in period_specs:

                if period_spec[:3] == "WWF" and wwf_plot_header_added == False:
                    f.write('<h2>10. WWF Calibration Plots</h2>\n')
                    wwf_plot_header_added = True
                                    

                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

                chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

                measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                
                if location_type == 'PS':
                    measured_gauge['Discharge_Hourly'] = measured_gauge['Flow'].rolling('1h').mean()
                
                measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                
                result_velocity_gauge = velocity_df_all.loc[(velocity_df_all.MUID==pipe) & (velocity_df_all.index >= start) & (velocity_df_all.index <= end)].copy()
                
                spill_df_zone = spill_df_zones.loc[(spill_df_zones.Zone==zone) & (spill_df_zones.Date_Time >= start) & (spill_df_zones.Date_Time <= end)].copy()                
                spill_df_all_zone = spill_df_all.loc[(spill_df_all.Zone==zone) & (spill_df_all.index >= start) & (spill_df_all.index <= end)].copy()
                spill_df_all_zone.drop(columns=['Zone'],inplace=True)
                spill_top_ten = spill_df_all_zone.groupby(['Node']).max()
                spill_top_ten.reset_index(inplace=True)
                spill_top_ten.sort_values(by='Spill',ascending=False,inplace=True)
                spill_top_ten.reset_index(drop=True,inplace=True)
                spill_top_ten = spill_top_ten[spill_top_ten['Spill']>0]
                spill_top_ten = spill_top_ten.head(10)                                
                
                if location_type == 'PS':
                    result_flow_gauge['Discharge_Hourly'] = result_flow_gauge['Discharge'].rolling('1h').mean()
                
                result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

                has_outfalls = False
                if zone in list(outfalls.Zone):
                    has_outfalls = True
                    outfalls_filter = outfalls[outfalls.Zone==zone].copy()
                    outfalls_filter.reset_index(inplace=True)

                    for index, row in outfalls_filter.iterrows():
                        overflow_pipe = row['Res_ID']
                        result_outfalls = flow_df_all.loc[(flow_df_all.MUID==overflow_pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                        result_outfalls['Outfall'] = row['Outfall']
                        if index == 0:
                            result_outfalls_all = result_outfalls.copy()
                        else:
                            result_outfalls_all = pd.concat([result_outfalls_all,result_outfalls])
                
                
            #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]
                #---------------------------------------------------------------------------------------------
                compare_stats = []
                peak_level_model = round(result_level_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Model',peak_level_model,'m'])
                peak_level_measured = round(measured_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Measured',peak_level_measured,'m'])
                compare_stats.append(['Peak Level Difference',round(peak_level_model - peak_level_measured,2),'m'])
                volume_model = round(result_flow_gauge.Volume.sum(),0).item()
                compare_stats.append(['Volume Model',volume_model,'m3'])
                volume_measured = round(measured_gauge.Volume.sum(),0).item() #convert to native float to avoid inf
                compare_stats.append(['Volume Measured',volume_measured,'m3'])
                try:
                    compare_stats.append(['Volume Difference',round((volume_model - volume_measured) / volume_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Volume Difference','','%'])
                peak_flow_model = round(result_flow_gauge.Discharge.max(),1)
                compare_stats.append(['Peak Flow Model',peak_flow_model,'L/s'])
                peak_flow_measured = round(measured_gauge.Flow.max(),1)
                compare_stats.append(['Peak Flow Measured',peak_flow_measured,'L/s'])
                try:
                    compare_stats.append(['Peak Flow Difference',round((peak_flow_model - peak_flow_measured) / peak_flow_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Peak Flow Difference','','%'])
                
                
                f.write('<h3>' + chart_header + '</h3>\n')  
                #----------------------------------------------------------------------------------------------------   
                if period_spec[:3] == "DWF":
                    
                    f.write('<div class="row">')
                    
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Description</th>\n')
                    f.write('    <th>Value</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')                   
                    
                    #sim and measured stats
                    for compare_stat in compare_stats:
                        f.write('  <tr>\n')
                        f.write('    <td>' + compare_stat[0] + '</th>\n')
                        f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                        f.write('    <td>' + compare_stat[2] + '</th>\n')
                        f.write('  </tr>\n')
                        
                    f.write('</table>\n')
                    f.write('</div>')                    
                    
                    #Loading rate
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Loading Rate</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')
                                       
                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 3
                        else: 
                            decimals = 1
                        rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                        unit = all_types.loc[ww_type,'Unit2']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(rate) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')

                    #Population and ICI areas
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    f.write('    <th>Total</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 0
                        else: 
                            decimals = 1
                        description = unit = all_types.loc[ww_type,'Description']
                        local_val = round(dwf_specs.loc[zone,ww_type + '_' + description],decimals)
                        total_val = round(dwf_specs.loc[zone,ww_type + '_' + description + '_Upstream'],decimals)
                        unit = all_types.loc[ww_type,'Unit1']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        f.write('    <td>' + str(total_val) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')
                    #Waterload
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    f.write('    <th>Total</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for load_type in all_types.index.values.tolist():

                        local_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad'],0)
                        total_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad_Upstream'],0)
                        unit = 'm3/d'

                        f.write('  <tr>\n')
                        f.write('    <td>' + load_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        f.write('    <td>' + str(total_val) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                        
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')

                    f.write('</table>\n')
                    f.write('</div>')
#                     f.write('</div>')
                    
                else:

                    total_rows = 10
                    f.write('<div class="row">')
                    for i in range(0,4): #Cycle through the three table types.   
                        f.write('<div class="column">')    
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Description</th>\n')
                        f.write('    <th>Value</th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        current_row = 0
                        if i == 0:
                            for compare_stat in compare_stats:
                                f.write('  <tr>\n')
                                f.write('    <td>' + compare_stat[0] + '</th>\n')
                                f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                                f.write('    <td>' + compare_stat[2] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1                
                        else:
                            for index, row in wwf_stats_specs[wwf_stats_specs.Table==i].iterrows():

                                decimals = 2

                                rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                                unit = all_types.loc[ww_type,'Unit2']


                                f.write('  <tr>\n')
                                f.write('    <td>' + row['Description'] + '</th>\n')
                                if row['DF'] == 'wwf_specs':
                                    f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']],2)) + '</th>\n')
                                else: 
                                    f.write('    <td>' + str(round(measured_rain[rain_gauge].sum(),1)) + '</th>\n')
                                f.write('    <td>' + row['Unit'] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1

                        remaining_rows = total_rows - current_row

                        for remaining_row in range(remaining_rows):
                            f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('</table>\n')
                        f.write('</div>')
                        
                #Spill top 10
                if period_spec[:3] == "DWF":
                    total_rows = 9
                else:
                    total_rows = 10
                    
                f.write('<div class="row">')
                  
                f.write('<div class="column">')    
                f.write('<table class="second">\n')
                f.write('  <tr>\n')
                f.write('    <th>Node Spill Top 10</th>\n')
                f.write('    <th>Peak Spill (L/s)</th>\n')
                f.write('  </tr>\n')

                current_row = 0

                for index, row in spill_top_ten.iterrows():
                    f.write('  <tr>\n')
                    f.write('    <td>' + row['Node'] + '</th>\n')
                    f.write('    <td>' + str(round(row['Spill'],0)) + '</th>\n')
                    f.write('  </tr>\n')
                    current_row += 1                


                remaining_rows = total_rows - current_row

                for remaining_row in range(remaining_rows):
                    f.write('<tr><td><br/></td><td><br/></td></tr>\n')
                f.write('</table>\n')
                f.write('</div>')
        

                f.write('</div>')
                    #----------------------------------------------------------------------------------------------


                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Dicharge at Gauge ' + gauge + ', Reach ' + pipe + ', Rain Gauge ' + rain_gauge,  
                                                    'Water Level at Gauge ' + gauge + ', Node ' + node),
                                                    specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                                                   horizontal_spacing = 0.1)

                fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)

                if location_type == 'PS':
                    fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge_Hourly, mode='lines',name='Model Flow Hourly',line_color='yellow'),1,1)   
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Discharge_Hourly, mode='lines',name='Measured Flow Hourly',line_color='black'),1,1)
                
                fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level',line_color='green'),1,2)
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level',line_color='purple'),1,2)
                           
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ),
                    title ={
                        'text' : chart_header,
                        'x':0.45,
                        'xanchor': 'center'
                    })
                
                fig['layout']['yaxis']['title']='Dicharge (L/s)'
                fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
                fig['layout']['yaxis2']['range']=[10,0]
                fig['layout']['yaxis3']['title']='Water Level (m)'
                
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')
                
                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Velocity at Gauge ' + gauge + ', Reach ' + pipe,
                                                    'Total Manhole Spills, CSO and SSO'),
                                                    specs=[[{"secondary_y": False}, {"secondary_y": False}]],
                                                   horizontal_spacing = 0.1)
                
                fig.add_trace(go.Scatter(x=result_velocity_gauge.DateTimeRef, y=result_velocity_gauge.Velocity, mode='lines',name='Model Velocity'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Velocity, mode='lines',name='Measured Velocity'),1,1)
                
                spill_df_zone
                fig.add_trace(go.Scatter(x=spill_df_zone.Date_Time, y=spill_df_zone.Spill, mode='lines',name='Total Manhole Spilling'),1,2)
                if has_outfalls == True:
                
                    for outfall in list(outfalls_filter.Outfall):

                        result_outfalls_all_filter = result_outfalls_all[result_outfalls_all.Outfall==outfall]

                        
                        fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, y = result_outfalls_all_filter.Discharge, mode='lines',name=outfall),1,2)
                                        
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ))
#                     title ={
#                         'text' : chart_header,
#                         'x':0.45,
#                         'xanchor': 'center'
#                     })
                
                fig['layout']['yaxis']['title']='Velocity (m/s)'
                fig['layout']['yaxis2']['title']='Dicharge (L/s)'
                
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')
                          

        f.close()
        print("Writing html for " + gauge)
#     except Exception as e:
#         error_list.append([gauge_id,e])

    

Writing html for FA2
Writing html for GN14
Writing html for HL14
Writing html for HL18
Writing html for HL1
Writing html for HL33
Writing html for HL94
Writing html for HL95
Writing html for HL97
Writing html for HL9
Writing html for MK1
Writing html for MK20
Writing html for MK3
Writing html for NV13
Writing html for NV19
Writing html for NV1
Writing html for NV20
Writing html for NV21
Writing html for NV31
Writing html for NV45
Writing html for NV8
Writing html for NV93
Writing html for NV94
Writing html for NV9
Writing html for NVLS1
Writing html for NVLS5-450
Writing html for NVLS5-500
Writing html for NVLS5-Blowdown
Writing html for NVLS9
Writing html for TW3B-MP1
Writing html for TW3B-MP2
Writing html for GE1
Writing html for GE2
Writing html for GE3
Writing html for GE4
Writing html for GE5
Writing html for LY1
Writing html for LY2
Writing html for LY7


In [431]:
#THIS IS A PERMANENT CELL
event_categories = ['DWF Calibration','WWF Calibration','WWF Validation']

result_types = ['HGL','Flow','Volume']

result_points = []
for index, row in map_periods[1:2].iterrows():
    gauge = row['Meter']
    zone = row['Zone']
    meter_status = row['Meter Status']
    x = gauges.loc[gauge,'X coordinate']
    y = gauges.loc[gauge,'Y coordinate']
    x_shift = gauges.loc[gauge,'Shift X (m)']
    y_shift = gauges.loc[gauge,'Shift Y (m)']
    catchment_type = gauges.loc[gauge,'Catchment Type']

    for i, event_category in enumerate(event_categories[:1]):
        event_category_short = event_category[:7]
        start = row[event_category_short + ' Start']
        end = row[event_category_short + ' End']
        y_minor_shift = -i * map_point_spacing
        y_final = y + y_shift + y_minor_shift

        color_values = []
        for j, result_type in enumerate(result_types):
            x_minor_shift = j * map_point_spacing
            x_final = x + x_shift + x_minor_shift
            bad_data = False
            if result_type == 'HGL':
                result_type_csv = 'Max HGL'
                qa = row[event_category_short + ' WL QA']
                if qa.lower() == 'bad data':
                    bad_data = True
                else:
                    node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
                    max_model = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)].Level.max()
                    max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Level.max()
                    if not math.isnan(max_measured) and max_measured != 0:
                        diff = abs(max_model - max_measured) * 100 #cm
                    else:
                        diff = 'NA'

            elif result_type == 'Flow':
                result_type_csv = 'Max Flow'
                qa = row[event_category_short + ' Flow QA']
                if qa.lower() == 'bad data':
                    bad_data = True
                else:
                    pipe = str(gauges.loc[gauge,'Pipe'])
                    filter_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                    filter_model['Hourly'] = filter_model.Discharge.rolling('1h').mean()
                    filter_model = filter_model.iloc[12: , :] #Remove first hour to not get skewed hourly average there                        
                    max_model = filter_model.Hourly.max()

                    filter_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                    filter_measured['Hourly'] = filter_measured.Flow.rolling('1h').mean()
                    filter_measured = filter_measured.iloc[12: , :] #Remove first hour to not get skewed hourly average there 
                    max_measured = filter_measured.Hourly.max()
                    if not math.isnan(max_measured) and max_measured != 0:
                        diff = abs(max_model - max_measured) / max_measured
                    else:
                        diff = 'NA'

            else:
                result_type_csv = 'Acc Volume'
                qa = row[event_category_short + ' Flow QA']
                if qa.lower() == 'bad data':
                    bad_data = True
                else:
                    pipe = str(gauges.loc[gauge,'Pipe'])
                    max_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].Volume.sum()
                    max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Volume.sum()
                    if not math.isnan(max_measured) and max_measured != 0:
                        diff = abs(max_model - max_measured) / max_measured
                    else:
                        diff = 'NA'

            if bad_data == True:
                color = 'Black'
                color_value = 0
            elif diff == 'NA':
                color = 'Grey'
                color_value = 0
            else:
                if event_category[:3] == 'WWF':
                    threshold_column = event_category[:3] + ' ' + catchment_type + ' ' + result_type + ' Status'
                else:
                    threshold_column = event_category[:3] + ' ' + result_type + ' Status'
                for k in range(3):
                    threshold = thresholds.loc[k,threshold_column]
                    if diff >= threshold:
                        color = thresholds.loc[k,'Color']
                        color_value = thresholds.loc[k,'index']
                        break
                        
            color_values.append(color_value)

            label = ''
            if i == 0 and j == 0:
                label = gauge

            result_points.append([gauge,x_final,y_final,label,event_category,result_type_csv,color])

        if meter_status == 'Primary':
            weighting_components = []
            weighted_color_value = 0
            value_count = 0
            for k, color_value in enumerate(color_values):
                if color_value > 0:
                    if k == 0:
                        weighting_components.append(color_value) #Level is weighted as 50%
                        value_count += 1
                    else:
                        weighting_components.append(color_value/2) #Flow is weighted as 25%
                        value_count += 0.5
            qa = 'OK'
            if len(weighting_components) == 0:
                weighted_color_value = 0
                qa = 'All Data Issue'
            else:
                sum_color_values = 0
                for weighting_component in weighting_components:
                    sum_color_values += weighting_component
                weighted_color_value = sum_color_values / value_count
                weighted_color_value = round((weighted_color_value+0.001)*2,0)/2 #2.25 should round up to 2.5 as in Excel
            print (zone + ' ' + str(weighted_color_value))
                        
result_points = pd.DataFrame(result_points,columns=['Gauge','X','Y','Label','SimType','ResultType','Color'])
result_points.to_csv(output_folder + '\\ResultPoints.csv',index=False)



GE1 2.5


In [430]:
round(2.51,0)

3.0

In [424]:
result_points

,Gauge,X,Y,Label,SimType,ResultType,Color,mmo,mme
0,FA2,493075.340271,5.464172e+06,FA2,DWF Calibration,Max HGL,Yellow,42.625920,42.546902
1,FA2,493076.340271,5.464172e+06,,DWF Calibration,Max Flow,Green,103.467433,105.645834
2,FA2,493077.340271,5.464172e+06,,DWF Calibration,Acc Volume,Green,26574.317460,26835.733483
3,FA2,493075.340271,5.464171e+06,,WWF Calibration,Max HGL,Green,42.852650,42.824600
4,FA2,493076.340271,5.464171e+06,,WWF Calibration,Max Flow,Green,361.082383,368.979167
5,FA2,493077.340271,5.464171e+06,,WWF Calibration,Acc Volume,Green,48080.116149,51442.557044
6,FA2,493075.340271,5.464170e+06,,WWF Validation,Max HGL,Grey,42.830660,NaN
7,FA2,493076.340271,5.464170e+06,,WWF Validation,Max Flow,Grey,328.403725,NaN
8,FA2,493077.340271,5.464170e+06,,WWF Validation,Acc Volume,Grey,63917.693103,0.000000


In [411]:

thresholds

,index,Color,DWF Flow Status,WWF SEP Flow Status,WWF CMB Flow Status,DWF HGL Status,WWF SEP HGL Status,WWF CMB HGL Status,DWF Volume Status,WWF SEP Volume Status,WWF CMB Volume Status
0,3,Red,0.15,0.2,0.3,15,20,20,0.15,0.2,0.3
1,2,Yellow,0.05,0.1,0.1,5,10,10,0.05,0.1,0.1
2,1,Green,0,0,0,0,0,0,0,0,0


In [346]:
max_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
# max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Flow.rolling('1h').mean().max()
max_model['rolling'] = max_model.Discharge.rolling('1h').mean()
max_model.to_csv(output_folder + '\\test.csv')

In [239]:
measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)]


                

,Gauge,Flow,Level,Velocity,Seconds,Volume
2018-05-18 00:00:00,FA2,74.540001,42.500198,1.335,300.0,22.362000
2018-05-18 00:05:00,FA2,72.385002,42.498600,1.320,300.0,21.715501
2018-05-18 00:10:00,FA2,69.614998,42.495300,1.310,300.0,20.884499
2018-05-18 00:15:00,FA2,66.195000,42.490898,1.300,300.0,19.858500
2018-05-18 00:20:00,FA2,64.285004,42.488201,1.300,300.0,19.285501
...,...,...,...,...,...,...
2019-05-07 23:35:00,FA2,88.830002,42.532200,1.215,300.0,26.649001
2019-05-07 23:40:00,FA2,86.035004,42.529202,1.205,300.0,25.810501
2019-05-07 23:45:00,FA2,85.110001,42.526798,1.215,300.0,25.533000
2019-05-07 23:50:00,FA2,82.925003,42.524399,1.210,300.0,24.877501


In [240]:
start


Timestamp('2020-01-29 00:00:00')

In [274]:
gauges

,Location,Pipe,Node1 (Or Pipe if pipe level),Node2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type,Catchment Type,X coordinate,Y coordinate,Shift X (m),Shift Y (m)
Gauge,,,,,,,,,,,,,,,,,,
FA2,FA2,49582,9600,NaN,DN72,NSSA,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node,SEP,493075.340271,5.464172e+06,0.0,0.0
GE1,GE1,50924,GE1-WetWell,NaN,VW67,NSSA,Level and Flow,PS,Circular,0.2,,Measured,Node,SEP,480113.346130,5.469160e+06,0.0,0.0
GE2,GE2,56296,GE2-WetWell,NaN,VW67,NSSA,Level and Flow,PS,Circular,0.2,NaN,Measured,Node,SEP,479877.110901,5.468782e+06,0.0,0.0
GE3,GE3,50933,GE3-WetWell,NaN,VW49,NSSA,Level and Flow,PS,Circular,0.25,,Measured,Node,SEP,480538.298706,5.466816e+06,0.0,0.0
GE4,GE5,50940,GE4-WetWell,NaN,VW49,NSSA,Level and Flow,PS,Circular,0.25,,Measured,Node,SEP,480661.689880,5.466720e+06,0.0,-300.0
GE5,GE5,50952,GE5-WetWell,NaN,VW49,NSSA,Level and Flow,PS,Circular,0.25,,Measured,Node,SEP,480661.689880,5.466720e+06,0.0,-300.0
GN14,GN14,49709,7169,NaN,VW67,NSSA,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node,SEP,480516.995117,5.467189e+06,0.0,0.0
HL1,HL1,43953,9721,NaN,VW14,NSSA,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node,SEP,489702.910095,5.463308e+06,0.0,-200.0
HL14,HL14,43945,9712,NaN,VW14,NSSA,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node,SEP,488765.324707,5.463838e+06,200.0,300.0


In [414]:
a = [1,2,3,4]
dir(a)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']